In [1]:
import pandas as pd
from time import sleep # control the crawl rate to avoid hammering the servers with too many requests
from random import randint
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities # dealing with pop-up permissions
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from scrapping import scrapping_foundations

from os import listdir

from google.cloud import storage
storage_client = storage.Client.from_service_account_json('foundation-matching-9bb2587b610a.json')

In [2]:
list_of_foundations = pd.read_csv('sephora_ProQuestionAsker.csv')
list_of_foundations_url = list_of_foundations.groupby(['brand', 'product'],as_index=False)['url'].first()

list_of_foundations_url['brand_product'] = ''
for i in tqdm(range(len(list_of_foundations_url))):
    list_of_foundations_url.loc[i, 'brand_product'] = list_of_foundations_url.loc[i, 'brand'].replace(' ', '_') + '_' + list_of_foundations_url.loc[i, 'product'].replace(' ', '_')

list_of_foundations_url = list_of_foundations_url[['brand_product', 'url']]

100%|██████████| 203/203 [00:00<00:00, 5027.12it/s]


In [3]:
geoBlocked = webdriver.FirefoxOptions()
geoBlocked.set_preference("geo.prompt.testing", True)
geoBlocked.set_preference("geo.prompt.testing.allow", False)
geoBlocked.set_preference("dom.push.enabled", False)
driver = webdriver.Firefox(options=geoBlocked)

url = list_of_foundations_url.loc[0, 'url']

driver.get(url)
sleep(3)
driver.execute_script(f"window.scrollTo(0, 2000)") 
sleep(2)
driver.execute_script(f"window.scrollTo(0, 3000)")

In [4]:
list_of_reviews = driver.find_elements(By.CSS_SELECTOR, 'div[data-comp="Review StyledComponent BaseComponent "]')

In [5]:
reviewer_id = []
rating = []
recommended = []
review_subject = []
review_content = []
reviewer_feature = []
purchased_shade = []
date_of_review = []

cols = {
    "reviewer_id": reviewer_id, "rating": rating, "recommended": recommended,
    "review_subject": review_subject, "review_content": review_content, 
    "reviewer_feature": reviewer_feature, "purchased_shade": purchased_shade, 
    "date_of_review": date_of_review
    }
data = pd.DataFrame(cols)

In [6]:
shade = ""
review_subject = ""
review_content = ""
reviewer_feature = ""
reviewer_id = ""
rating = int()
recommended = ""
date_of_review = ""
for i in list_of_reviews:

    if i.find_elements(By.CSS_SELECTOR, 'strong[data-at="nickname"]') != []:
        reviewer_id = i.find_elements(By.CSS_SELECTOR,'strong[data-at="nickname"]')[0].text
    else:
        pass 

    if i.find_elements(By.CSS_SELECTOR,'strong[data-at="nickname"]+span') != []:
        reviewer_feature = i.find_elements(By.CSS_SELECTOR,'strong[data-at="nickname"]+span')[0].text
    else:
        pass

    if i.find_elements(By.CSS_SELECTOR,'div[data-comp="StarRating "]') != []:
        rating = int(i.find_elements(By.CSS_SELECTOR, 'div[data-comp="StarRating "]')[0].get_attribute("aria-label")[0])
    else:
        pass

    if i.find_elements(By.CSS_SELECTOR, 'h3') != []:
        review_subject = i.find_element(By.CSS_SELECTOR, 'h3').text
    else:
        review_subject = None

    if i.find_elements(By.CSS_SELECTOR, 'img[src*="https://www.sephora.com/productimages/"]+span') != []:
        shade = i.find_elements(By.CSS_SELECTOR, 'img[src*="https://www.sephora.com/productimages/"]+span')[0].text
    else:
        shade = None
    
    if shade and i.find_elements(By.CSS_SELECTOR, 'h3+div+div') != []:
        review_content = i.find_element(By.CSS_SELECTOR, 'h3+div+div').text
    if shade is None and i.find_elements(By.CSS_SELECTOR, 'h3+div') != []:
        review_content = i.find_element(By.CSS_SELECTOR, 'h3+div').text
    else:
        pass

    if i.find_elements(By.CSS_SELECTOR, 'span[data-at="time_posted"]') != []:
        date_of_review = i.find_element(By.CSS_SELECTOR, 'span[data-at="time_posted"]').text
    else:
        date_of_review = None

    if 'Recommended' in i.find_element(By.CSS_SELECTOR, 'div').get_attribute('innerHTML'):
        recommended = 1
    else:
        recommended = 0

new_col = {
    "reviewer_id": reviewer_id,
    "rating": rating,
    "recommended": recommended,
    "review_subject": review_subject,
    "reivew_content": review_content,
    "reviewer_feature": reviewer_feature,
    "purchased_shade": shade,
    "date_of_review": date_of_review
}

In [7]:
data = data.append(new_col,ignore_index=True)

In [8]:
data

,reviewer_id,rating,recommended,review_subject,review_content,reviewer_feature,purchased_shade,date_of_review,reivew_content
0,,3.0,0.0,Overly fragranced!,NaN,,204 Light/Medium Yellow - light to medium skin...,31 Jul 2021,204 Light/Medium Yellow - light to medium skin...
